In [329]:
import os
import pymysql
import pandas as pd
import altair as alt

## Connecting to Database

In [237]:
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = "cricket123"
database = os.getenv('MYSQL_DATABASE')

In [238]:
conn = pymysql.connect(
    host = host,
    port = port,
    user = "root",
    passwd = password,
    db = "happiness",
    charset = 'utf8mb4')

## Question #1

In [248]:
starting_year = "2015"
ending_year = "2018"
country = "Switzerland"
query1 = "SELECT happiness.`" + starting_year + "`.`country`, happiness.`" + \
          starting_year + "`.`Happiness Rank` - happiness.`" + ending_year + \
          "`.`Happiness Rank` AS 'Happiness Rank Difference' FROM happiness.`" + \
          ending_year + "` INNER JOIN happiness.`" + starting_year + \
          "` ON happiness.`" + ending_year + "`.`Country` = happiness.`" + starting_year + \
          "`.`Country` WHERE happiness.`" + ending_year + "`.`Country` = '" + country + "'"
df1 = pd.read_sql_query(query1, conn)
df1

/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country,Happiness Rank Difference
0,Switzerland,-4


## Question #2

### Finding the average rank difference amongst countries in a region against two different years

In [249]:
starting_year = "2015"
ending_year = "2018"
region = "North America"
query2 = "SELECT happiness.`" + starting_year + "`.`region`, AVG(happiness.`" + \
          starting_year + "`.`Happiness Rank` - happiness.`" + ending_year + \
          "`.`Happiness Rank`) AS 'Happiness Rank Difference' FROM happiness.`" + \
          ending_year + "` INNER JOIN happiness.`" + starting_year + "` ON happiness.`" + \
          ending_year + "`.`Country` = happiness.`" + starting_year + "`.`Country` WHERE happiness.`" + \
          starting_year + "`.`region` = '" + region + "' GROUP BY `" + starting_year + "`.`region`"
df2 = pd.read_sql_query(query2, conn)
df2

/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,region,Happiness Rank Difference
0,North America,-2.5


## Question 3

### Out of curiosity, I wanted to see the breakdown of the factors that impacts the Happiness Rank and Score

In [331]:
starting_year = "2015"
ending_year = "2016"
country = 'Switzerland'
query3 = "SELECT happiness.`" + starting_year + "`.`country`,(happiness.`" + starting_year + \
"`.`Happiness Rank` - happiness.`" + ending_year + "`.`Happiness Rank`) AS 'Happiness Rank Difference', ROUND(happiness.`" \
+ starting_year + "`.`Happiness Score` - happiness.`" + ending_year + \
"`.`Happiness Score`, 4) AS 'Happiness Score Difference', ROUND(happiness.`" + starting_year + \
"`.`Economy` - happiness.`" + ending_year + "`.`Economy`, 4) AS 'Economic Difference', ROUND(happiness.`" + \
starting_year + "`.`Health` - happiness.`2016`.`Health`, 4) AS 'Health Difference', ROUND(happiness.`" + \
starting_year + "`.`Freedom` - happiness.`" + ending_year + "`.`Freedom`, 4) AS 'Freedom Difference', ROUND(happiness.`" + \
starting_year + "`.`Government` - happiness.`2016`.`Government`, 4) AS 'Governmental Difference' FROM happiness.`" + \
ending_year + "`INNER JOIN happiness.`" + starting_year + "`ON happiness.`" + \
ending_year + "`.`Country` = happiness.`" + starting_year + "`.`Country` WHERE `" + starting_year + "`.`Country` = 'Switzerland'"

diff = pd.read_sql_query(query3, conn)
diff

/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country,Happiness Rank Difference,Happiness Score Difference,Economic Difference,Health Difference,Freedom Difference,Governmental Difference
0,Switzerland,-1,0.078,-0.1308,0.0784,0.08,0.0077


### Interactive Plot: to visualize difference in economy, health, freedom, and government
#### - click left key for individual categories and hold shift for multiple - 

In [332]:
country = 'Switzerland'
starting_year = "2015"
ending_year = "2016"
start = "SELECT `Country`, `Happiness Rank`, `Happiness Score`, `Economy`, `Health`, `Freedom`, `Government` FROM `Happiness`.`" + starting_year + "` WHERE `Country` = '" + country + "'"
end = "SELECT `Country`, `Happiness Rank`, `Happiness Score`, `Economy`, `Health`, `Freedom`, `Government` FROM `Happiness`.`" + ending_year + "` WHERE `Country` = '" + country + "'"

start_df = pd.read_sql_query(start, conn)
start_df['Year'] = start_df['Year'] = starting_year
end_df = pd.read_sql_query(end, conn)
end_df['Year'] = end_df['Year'] = ending_year

all_df = pd.concat([start_df, end_df])
new_df = pd.melt(all_df, id_vars=['Year'], value_vars=['Economy', 'Health', 'Freedom', 'Government'], var_name = "Category")

category = pd.DataFrame({'Category': ['Economy', 'Health', 'Freedom', 'Government']})
selection = alt.selection_multi(fields=['Category'])
color = alt.condition(selection, alt.Color('Category:N'), alt.value('lightgray'))
make_selector = alt.Chart(category).mark_rect().encode(y='Category', color=color).add_selection(selection)
fuel_chart = alt.Chart(new_df).mark_line().encode(x='Year:N', y=alt.Y('value:Q', scale=alt.Scale(domain=[0, 2])), color='Category').transform_filter(selection).properties(width = 200)

make_selector | fuel_chart


/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


alt.HConcatChart(...)

## Streamlit App Misc.

### In order to create a dropdown menu with the different countries and regions options, below is how I got a list of countries/regions

In [200]:
country = pd.read_sql_query("SELECT DISTINCT `Country` FROM `happiness`.`2015` ORDER BY `Country` ASC;", conn)
countries_list = country['Country'].to_list()
print(countries_list)

['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Ivory Coast', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania', 'Luxembourg', 'Macedoni

/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [201]:
region = pd.read_sql_query("SELECT DISTINCT `Region` FROM `happiness`.`2015` ORDER BY `Region` ASC;", conn)
regions_list = region['Region'].to_list()
print(regions_list)

['Australia and New Zealand', 'Central and Eastern Europe', 'Eastern Asia', 'Latin America and Caribbean', 'Middle East and Northern Africa', 'North America', 'Southeastern Asia', 'Southern Asia', 'Sub-Saharan Africa', 'Western Europe']


/Users/kelkong/opt/miniconda3/lib/python3.9/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
